Connected to .venv (Python 3.10.16)

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  


#Optional: Artificially mask. Mask 20% of the data (MIT part), try 30% to compare with GP-VAE
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS_2")
#SAITS_run_name = "SAITS_1"


# Define comparison plot function

def plot_comparison(original, masked, imputed, num_samples=3, num_features=3):
    sample_indices = np.random.choice(original.shape[0], num_samples, replace=False)
    feature_indices = np.random.choice(original.shape[2], num_features, replace=False)

    fig, axes = plt.subplots(num_samples, num_features, figsize=(5 * num_features, 4 * num_samples))

    for i, sample_idx in enumerate(sample_indices):
        for j, feature_idx in enumerate(feature_indices):
            ax = axes[i, j] if num_samples > 1 else axes[j]
            ax.plot(original[sample_idx, :, feature_idx], label='Original', color='blue')
            ax.plot(masked[sample_idx, :, feature_idx], label='Masked', color='orange', linestyle='dashed')
            ax.plot(imputed[sample_idx, :, feature_idx], label='Imputed', color='green')
            ax.set_title(f'Sample {sample_idx}, Feature {feature_idx}')
            ax.legend()

    plt.tight_layout()
    plt.show()

# Optuna objective function
def objective(trial):
    params = {
        "n_layers": trial.suggest_int("n_layers", 2, 4),
        "d_model": trial.suggest_categorical("d_model", [64, 128, 256]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 20),
        "batch_size": trial.suggest_int("batch_size", 4, 16)
    }

    with mlflow.start_run(run_name="SAITS_testcomparedtoGPVAE", nested=True):
        mlflow.log_params(params)

        saits = SAITS(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            n_layers=params["n_layers"],
            d_model=params["d_model"],
            optimizer=Adam(lr=params["lr"]),
            ORT_weight=1.0,
            MIT_weight=1.0,
            batch_size=params["batch_size"],
            epochs=params["epochs"],
            d_ffn=512,
            n_heads=8,
            d_k=64,
            d_v=64,
            dropout=0.1,
            attn_dropout=0.1,
            diagonal_attention_mask=True,
            patience=6,
            num_workers=0,
            device=device,
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
            model_saving_strategy="best",
        )

        saits.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        test_imputation = saits.predict({"X": test_X_masked})["imputation"]
        test_imputation_denorm = inverse_scale(test_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)
        

           # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse_i)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", avg_mae)
        mlflow.log_metric("avg_rmse", avg_rmse)

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("SAITS-2")
with mlflow.start_run(run_name="SAITS_Optuna_Study") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]
       

    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

print("Best Parameters:", best_params)
print("Best Objective Value:", best_value)



# Re-run the model with best parameters
saits_best = SAITS(
    n_steps=data_reshaped.shape[1],
    n_features=data_reshaped.shape[2],
    n_layers=best_params["n_layers"],
    d_model=best_params["d_model"],
    optimizer=Adam(lr=best_params["lr"]),
    ORT_weight=1.0,
    MIT_weight=1.0,
    batch_size=best_params["batch_size"],
    epochs=best_params["epochs"],
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,
    patience=6,
    num_workers=0,
    device=device,
    saving_path="...",  # optional: path to save best model
    model_saving_strategy="best",
)

saits_best.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
test_imputation_best = saits_best.predict({"X": test_X_masked})["imputation"]
test_imputation_best_denorm = inverse_scale(test_imputation_best, scalers)
test_ori_denorm = inverse_scale(test_X_ori, scalers)

# Plot the comparison
plot_comparison(test_ori_denorm, test_X_masked, test_imputation_best_denorm)
###Till here!!!

2025-05-20 12:31:13 [WARNING]: ‼️ PyPOTS Ecosystem configuration file does not exist.
2025-05-20 12:31:13 [INFO]: Wrote new configs to config.ini successfully.
2025-05-20 12:31:13 [INFO]: 💫 Initialized PyPOTS Ecosystem configuration file /home/ec2-user/.pypots/config.ini successfully.
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

CUDA available: True
Using CUDA


2025/05/20 12:31:29 INFO mlflow.tracking.fluent: Experiment with name 'SAITS_2' does not exist. Creating a new experiment.
2025/05/20 12:31:29 INFO mlflow.tracking.fluent: Experiment with name 'SAITS-2' does not exist. Creating a new experiment.
[I 2025-05-20 12:31:29,860] A new study created in memory with name: no-name-bd0fbb77-ab84-4d60-9adb-d99753d60f5f
2025-05-20 12:31:29 [INFO]: Using the given device: cuda
2025-05-20 12:31:29 [INFO]: Model files will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250520_T123129
2025-05-20 12:31:29 [INFO]: Tensorboard file will be saved to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250520_T123129/tensorboard
2025-05-20 12:31:29 [INFO]: Using customized MAE as the training loss function.
2025-05-20 12:31:29 [INFO]: Using customized MSE as the validation metric function.
2025-05-20 12:31:29 [WARNING]: ‼️ d_model must = n_heads * d

🏃 View run SAITS_testcomparedtoGPVAE at: http://localhost:5000/#/experiments/9/runs/512c8eb4f2944d00bc4ee2bdfe8e923d
🧪 View experiment at: http://localhost:5000/#/experiments/9
🏃 View run SAITS_Optuna_Study at: http://localhost:5000/#/experiments/9/runs/2bbab57f67ca4e3a966d70fc4bdf2a87
🧪 View experiment at: http://localhost:5000/#/experiments/9


NameError: name 'run' is not defined

In [ ]:
#Import Pypots Library
from pypots.optim import Adam
from pypots.imputation import SAITS
#from pypots.utils.metrics import calc_mae
from pypots.nn.functional import calc_mae


import argparse
import hashlib
from pathlib import Path

import matplotlib.pyplot as plt
import mlflow
import mlflow.pytorch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.utils.data
import data_insight
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from duckdb import DuckDBPyConnection as DuckDB
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna 
from optuna.visualization import plot_optimization_history




from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, Dataset
from pygrinder.missing_completely_at_random import mcar
from tqdm.auto import tqdm

import sensor_imputation_thesis.shared.load_data as load

torch.cuda.empty_cache()
#PatchTST might be an ideal choise if SAITS is too slow 

##Drop columns with different indexes while loading data.. Or the mean values 

df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/ny_df_for_pypots.parquet")

len(df)

#current length of the dataframe is 119439

# Check nan values in each column
for col in df.columns:
    print(f"Column {col} has {df[col].isna().sum()} NaN values")
    missing_rate=df[col].isna().sum()/len(df[col])
    print(f"Column {col} has {missing_rate} Missing_rate")


#Try with smaller dataset, size 4000
##SAMPLE the percengtage of the dataset, df.sample (averagely pick samples)
#not df.sample cuz it will randomly select 
original_size=len(df)
desired_fraction=0.3 #Select data every 3 minutes 
step=int(1/desired_fraction) #step_size=10 (sample every 10th (3/10) minute)

#Systematic sampling: Start at a random offset to avoid bias 
start=np.random.randint(0,step) #Random start between 0-9
df1=df.iloc[start::step].reset_index(drop=True)

print(f"Original size:{len(df)}, Sampled size: {len(df1)}")



# Custom Dataset class
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Data processing code
sensor_cols = [col for col in df1.columns if col != "time"]
data = df1[sensor_cols].values

#¤get feature names for printing mae later 
feature_names=df1[sensor_cols].columns.tolist()

## Convert data to 3D arrays of shape n_samples, n_timesteps, n_features, X_ori refers to the original data without missing values 
## Reconstruct all columns simultaneously  #num_features: 119
n_features = data.shape[1]  # exclude the time column
n_steps = 20 #60 (was 60 previously) #(TRY TO CHANGE HERE)  # # window length, 1440 steps = 24 hours of 1-minute data, but here is revised to 60 again
#total_elements = data.shape[0] * data.shape[1]
n_samples = data.shape[0] // n_steps 



# Reshape to (n_samples // n_steps, n_steps, n_features)
#data_reshaped = data.reshape((n_samples, n_steps, n_features))
data_reshaped=data[:n_samples*n_steps].reshape(n_samples,n_steps,n_features)
print(f"Reshaped data:{data.shape}")

#Split into train, test, val, fit scaler only on the train set (prevent data leakage)

#train_size = int(0.6 * len(data))
#val_size = int(0.2 * len(data))
#test_size = len(data) - train_size - val_size

#train_data = data_reshaped[:train_size]
#val_data = data_reshaped[train_size:train_size + val_size]
#test_data= data_reshaped[train_size + val_size:]


#Apply time series split 
#Split into train(60%), val(20%), and test (20%)
train_data, temp_data=train_test_split(data_reshaped,test_size=0.4,shuffle=True)
val_data, test_data=train_test_split(temp_data, test_size=0.5, shuffle=False)

##Normalization is important because of the nature of mse calculation of saits, columns with large 
#values dominate the loss, making metrics meaningless. SAITS computes MSE/MAE column-wise and averages 
#them across all columns 
#  Apply minmax scaler here 
#normalize each feature independently
scalers={}


#train_scaled = np.zeros_like(data_reshaped[train_size])  # Initialize the normalized data array
#val_scaled=np.zeros_like(data_reshaped[train_size:train_size + val_size])
#test_scaled=np.zeros_like(data_reshaped[train_size + val_size:])

train_scaled = np.zeros_like(train_data)
val_scaled = np.zeros_like(val_data)
test_scaled = np.zeros_like(test_data)



for i in range(data_reshaped.shape[2]):
    scaler = MinMaxScaler(feature_range=(-1, 1)) #changed to -1,1
    # Flatten timesteps and samples for scaling
    train_scaled[:, :, i] = scaler.fit_transform(train_data[:, :, i].reshape(-1, 1)).reshape(train_data.shape[0], train_data.shape[1])
    val_scaled[:, :, i] = scaler.transform(val_data[:, :, i].reshape(-1, 1)).reshape(val_data.shape[0], val_data.shape[1])
    test_scaled[:, :, i] = scaler.transform(test_data[:, :, i].reshape(-1, 1)).reshape(test_data.shape[0], test_data.shape[1])
    scalers[i] = scaler  # Save scalers to inverse-transform later

#Inverse Scale
def inverse_scale(imputation, scalers):
    n_features = imputation.shape[2]
    imputation_denorm = np.empty_like(imputation)
    
    for i in range(n_features):
        imputation_denorm[:, :, i] = scalers[i].inverse_transform(imputation[:, :, i].reshape(-1, 1)).reshape(imputation.shape[0], imputation.shape[1])
    
    return imputation_denorm  


#Optional: Artificially mask. Mask 20% of the data (MIT part), try 30% to compare with GP-VAE
def mcar_f(X, mask_ratio=0.3):
    """Apply MCAR only to observed values."""
    observed_mask=~np.isnan(X) #find observed positions
    artificial_mask=mcar(X,mask_ratio).astype(bool) #generate MCAR mask, cast to boolean
    #combine masks 
    combined_mask=observed_mask & artificial_mask

    #Apply masking
    X_masked=X.copy()
    X_masked[combined_mask]=np.nan
    return X_masked,combined_mask


#Use mcar on validation data 
val_X_masked, val_mask =mcar_f(val_scaled)
val_X_ori=val_scaled.copy() 

test_X_masked, test_mask =mcar_f(test_scaled)
test_X_ori=test_scaled.copy() 


class Config:
    no_cuda = False
    no_mps = False
    seed = 1

args=Config()

torch.manual_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.seed)
np.random.seed(args.seed)


args.cuda = not args.no_cuda and torch.cuda.is_available()
use_mps = not args.no_mps and torch.backends.mps.is_available()

args.cuda = not args.no_cuda and torch.cuda.is_available()
print("CUDA available:", torch.cuda.is_available())


if args.cuda:
    device = torch.device("cuda")
    print("Using CUDA")
elif use_mps:
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

train_scaled = torch.tensor(train_scaled, dtype=torch.float32)
val_X_masked = torch.tensor(val_X_masked, dtype=torch.float32)
val_X_ori = torch.tensor(val_X_ori, dtype=torch.float32)

train_scaled = train_scaled.to(device)
val_X_masked = val_X_masked.to(device)
val_X_ori = val_X_ori.to(device)


#MLflow set up
mlflow.set_tracking_uri("http://localhost:5000")
client = mlflow.tracking.MlflowClient()
mlflow.set_experiment("SAITS_2")
#SAITS_run_name = "SAITS_1"


# Define comparison plot function

def plot_comparison(original, masked, imputed, num_samples=3, num_features=3):
    sample_indices = np.random.choice(original.shape[0], num_samples, replace=False)
    feature_indices = np.random.choice(original.shape[2], num_features, replace=False)

    fig, axes = plt.subplots(num_samples, num_features, figsize=(5 * num_features, 4 * num_samples))

    for i, sample_idx in enumerate(sample_indices):
        for j, feature_idx in enumerate(feature_indices):
            ax = axes[i, j] if num_samples > 1 else axes[j]
            ax.plot(original[sample_idx, :, feature_idx], label='Original', color='blue')
            ax.plot(masked[sample_idx, :, feature_idx], label='Masked', color='orange', linestyle='dashed')
            ax.plot(imputed[sample_idx, :, feature_idx], label='Imputed', color='green')
            ax.set_title(f'Sample {sample_idx}, Feature {feature_idx}')
            ax.legend()

    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()

# Optuna objective function
def objective(trial):
    params = {
        "n_layers": trial.suggest_int("n_layers", 2, 4),
        "d_model": trial.suggest_categorical("d_model", [64, 128, 256]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 20),
        "batch_size": trial.suggest_int("batch_size", 4, 16)
    }

    with mlflow.start_run(run_name="SAITS_testcomparedtoGPVAE", nested=True) as run:
        mlflow.log_params(params)

        saits = SAITS(
            n_steps=data_reshaped.shape[1],
            n_features=data_reshaped.shape[2],
            n_layers=params["n_layers"],
            d_model=params["d_model"],
            optimizer=Adam(lr=params["lr"]),
            ORT_weight=1.0,
            MIT_weight=1.0,
            batch_size=params["batch_size"],
            epochs=params["epochs"],
            d_ffn=512,
            n_heads=8,
            d_k=64,
            d_v=64,
            dropout=0.1,
            attn_dropout=0.1,
            diagonal_attention_mask=True,
            patience=6,
            num_workers=0,
            device=device,
            saving_path="/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model",
            model_saving_strategy="best",
        )

        saits.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
        test_imputation = saits.predict({"X": test_X_masked})["imputation"]
        test_imputation_denorm = inverse_scale(test_imputation, scalers)
        test_ori_denorm = inverse_scale(test_X_ori, scalers)
        

           # Calculate metrics
        mae_per_feature = []
        rmse_per_feature=[]
        percentage_mae_per_feature = []

        for i in range(n_features):
            imputation_i = test_imputation_denorm[:, :, i]
            ground_truth_i = test_ori_denorm[:, :, i]
            mask_i = test_mask[:, :, i]
            if np.isnan(imputation_i).any() or np.isnan(ground_truth_i).any():
                continue
            mae_i = calc_mae(imputation_i, ground_truth_i, mask_i)
            mae_per_feature.append(mae_i)
            rmse_i = np.sqrt(mean_squared_error(imputation_i, ground_truth_i))
            rmse_per_feature.append(rmse_i)

            #Calculate the original standard deviation for the feature
            std_dev_i = np.std(ground_truth_i[mask_i == 1])
             # Calculate the percentage of MAE relative to the standard deviation   
            if std_dev_i != 0:
                percentage_mae_i = (mae_i / std_dev_i) * 100
                percentage_mae_per_feature.append(percentage_mae_i)
            else:
                 percentage_mae_i = float('inf')
            
            mlflow.log_metric(f"MAE_{feature_names[i]}", mae_i)
            mlflow.log_metric(f"RMSE_{feature_names[i]}",rmse_i)
            mlflow.log_metric(f"Percentage_MAE_{feature_names[i]}", percentage_mae_i)

        avg_mae = np.mean(mae_per_feature)
        avg_rmse=np.mean(rmse_per_feature)
       
        mlflow.log_metric("avg_mae", avg_mae)
        mlflow.log_metric("avg_rmse", avg_rmse)

        trial.set_user_attr("mlflow_run_id", run.info.run_id)

        return avg_mae

    print("MAE per feature:", mae_per_feature)
    print("RMSE per feature",rmse_per_feature)
    print("Percentage MAE per feature:", percentage_mae_per_feature)
   

# Run Optuna study
mlflow.set_experiment("SAITS-2")
with mlflow.start_run(run_name="SAITS_Optuna_Study") as parent_run:
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)

    best_params = study.best_trial.params
    best_value = study.best_trial.value
    best_run_id = study.best_trial.user_attrs["mlflow_run_id"]
       

    # Log best parameters
    mlflow.log_params(best_params)

    # Log best metric(s)
    mlflow.log_metric("best_objective_value", best_value)
    mlflow.log_param("best_run_id", best_run_id)

print("Best Parameters:", best_params)
print("Best Objective Value:", best_value)



# Re-run the model with best parameters
saits_best = SAITS(
    n_steps=data_reshaped.shape[1],
    n_features=data_reshaped.shape[2],
    n_layers=best_params["n_layers"],
    d_model=best_params["d_model"],
    optimizer=Adam(lr=best_params["lr"]),
    ORT_weight=1.0,
    MIT_weight=1.0,
    batch_size=best_params["batch_size"],
    epochs=best_params["epochs"],
    d_ffn=512,
    n_heads=8,
    d_k=64,
    d_v=64,
    dropout=0.1,
    attn_dropout=0.1,
    diagonal_attention_mask=True,
    patience=6,
    num_workers=0,
    device=device,
    saving_path="...",  # optional: path to save best model
    model_saving_strategy="best",
)

saits_best.fit(train_set={"X": train_scaled}, val_set={"X": val_X_masked, "X_ori": val_X_ori})
test_imputation_best = saits_best.predict({"X": test_X_masked})["imputation"]
test_imputation_best_denorm = inverse_scale(test_imputation_best, scalers)
test_ori_denorm = inverse_scale(test_X_ori, scalers)

# Plot the comparison
save_path = "/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/comparison_plot.png"
plot_comparison(test_ori_denorm, test_X_masked, test_imputation_best_denorm,save_path)

Column time has 0 NaN values
Column time has 0.0 Missing_rate
Column fr_eng has 0 NaN values
Column fr_eng has 0.0 Missing_rate
Column te_exh_cyl_out__0 has 0 NaN values
Column te_exh_cyl_out__0 has 0.0 Missing_rate
Column pd_air_ic__0 has 0 NaN values
Column pd_air_ic__0 has 0.0 Missing_rate
Column pr_exh_turb_out__0 has 316581 NaN values
Column pr_exh_turb_out__0 has 1.0 Missing_rate
Column te_air_ic_out__0 has 0 NaN values
Column te_air_ic_out__0 has 0.0 Missing_rate
Column te_seawater has 0 NaN values
Column te_seawater has 0.0 Missing_rate
Column te_air_comp_in_a__0 has 316581 NaN values
Column te_air_comp_in_a__0 has 1.0 Missing_rate
Column te_air_comp_in_b__0 has 316581 NaN values
Column te_air_comp_in_b__0 has 1.0 Missing_rate
Column fr_tc__0 has 316581 NaN values
Column fr_tc__0 has 1.0 Missing_rate
Column pr_baro has 0 NaN values
Column pr_baro has 0.0 Missing_rate
Column pd_air_ic__0_1 has 0 NaN values
Column pd_air_ic__0_1 has 0.0 Missing_rate
Column pr_exh_rec has 0 NaN va

/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
/home/ec2-user/SageMaker/sensor-imputation-thesis/.venv/lib/python3.10/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return

🏃 View run SAITS_testcomparedtoGPVAE at: http://localhost:5000/#/experiments/9/runs/ee55b28bd714433baa18a8a80a649525
🧪 View experiment at: http://localhost:5000/#/experiments/9


2025-05-20 12:38:41 [INFO]: Epoch 001 - training loss (MAE): 0.2999, validation MSE: 0.1477
2025-05-20 12:38:47 [INFO]: Epoch 002 - training loss (MAE): 0.1716, validation MSE: 0.1565
2025-05-20 12:38:53 [INFO]: Epoch 003 - training loss (MAE): 0.1389, validation MSE: 0.1510
2025-05-20 12:38:59 [INFO]: Epoch 004 - training loss (MAE): 0.1213, validation MSE: 0.1529
2025-05-20 12:39:05 [INFO]: Epoch 005 - training loss (MAE): 0.1073, validation MSE: 0.1510
2025-05-20 12:39:11 [INFO]: Epoch 006 - training loss (MAE): 0.1007, validation MSE: 0.1533
2025-05-20 12:39:17 [INFO]: Epoch 007 - training loss (MAE): 0.0943, validation MSE: 0.1551
2025-05-20 12:39:17 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-05-20 12:39:17 [INFO]: Finished training. The best model is from epoch#1.
2025-05-20 12:39:17 [INFO]: Saved the model to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250520_T123835/SAITS.pypots
[I 20

🏃 View run SAITS_testcomparedtoGPVAE at: http://localhost:5000/#/experiments/9/runs/a3d7c6a522bf4a858069fb1d7e696921
🧪 View experiment at: http://localhost:5000/#/experiments/9


...

🏃 View run SAITS_testcomparedtoGPVAE at: http://localhost:5000/#/experiments/9/runs/b43fb8fc67fe4d278877e3a3ebcccf6d
🧪 View experiment at: http://localhost:5000/#/experiments/9


2025-05-20 13:14:03 [INFO]: Epoch 001 - training loss (MAE): 0.3106, validation MSE: 0.1562
2025-05-20 13:14:16 [INFO]: Epoch 002 - training loss (MAE): 0.1784, validation MSE: 0.1901
2025-05-20 13:14:30 [INFO]: Epoch 003 - training loss (MAE): 0.1459, validation MSE: 0.2281
2025-05-20 13:14:43 [INFO]: Epoch 004 - training loss (MAE): 0.1501, validation MSE: 0.2678
2025-05-20 13:14:57 [INFO]: Epoch 005 - training loss (MAE): 0.1458, validation MSE: 0.1657
2025-05-20 13:15:10 [INFO]: Epoch 006 - training loss (MAE): 0.1587, validation MSE: 0.1800
2025-05-20 13:15:24 [INFO]: Epoch 007 - training loss (MAE): 0.2551, validation MSE: 0.1929
2025-05-20 13:15:24 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-05-20 13:15:24 [INFO]: Finished training. The best model is from epoch#1.
2025-05-20 13:15:24 [INFO]: Saved the model to /home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/best_model/20250520_T131349/SAITS.pypots
[I 20

🏃 View run SAITS_testcomparedtoGPVAE at: http://localhost:5000/#/experiments/9/runs/8954f8756cc9471885ec0a68eea7feab
🧪 View experiment at: http://localhost:5000/#/experiments/9
🏃 View run SAITS_Optuna_Study at: http://localhost:5000/#/experiments/9/runs/deb91e26635c482da1538dd697322df8
🧪 View experiment at: http://localhost:5000/#/experiments/9
Best Parameters: {'n_layers': 3, 'd_model': 64, 'lr': 0.000991619557792019, 'epochs': 11, 'batch_size': 7}
Best Objective Value: 7663.0864225322075


2025-05-20 13:15:38 [INFO]: Epoch 001 - training loss (MAE): 0.3607, validation MSE: 0.1621
2025-05-20 13:15:50 [INFO]: Epoch 002 - training loss (MAE): 0.2865, validation MSE: 0.3570
2025-05-20 13:16:02 [INFO]: Epoch 003 - training loss (MAE): 0.4582, validation MSE: 0.1586
2025-05-20 13:16:13 [INFO]: Epoch 004 - training loss (MAE): 0.5057, validation MSE: 0.1660
2025-05-20 13:16:25 [INFO]: Epoch 005 - training loss (MAE): 0.5106, validation MSE: 0.1751
2025-05-20 13:16:36 [INFO]: Epoch 006 - training loss (MAE): 0.5345, validation MSE: 0.1634
2025-05-20 13:16:48 [INFO]: Epoch 007 - training loss (MAE): 0.5131, validation MSE: 0.1767
2025-05-20 13:16:59 [INFO]: Epoch 008 - training loss (MAE): 0.5053, validation MSE: 0.1739
2025-05-20 13:17:11 [INFO]: Epoch 009 - training loss (MAE): 0.5034, validation MSE: 0.1617
2025-05-20 13:17:11 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-05-20 13:17:11 [INFO]: Finished training. The best model is from epoc

ValueError: invalid literal for int() with base 10: '/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/comparison_plot.png'

In [ ]:
calculator

NameError: name 'calculator' is not defined

In [ ]:
100000/20

5000.0

In [ ]:
df["te_seawater"].std()

1.7053052591422315e-13

In [ ]:
df["te_seawater"]

300       273.15
301       273.15
302       273.15
303       273.15
304       273.15
           ...  
438730    273.15
438731    273.15
438732    273.15
438733    273.15
438734    273.15
Name: te_seawater, Length: 316581, dtype: float64

In [ ]:
df.describe()

,time,fr_eng,te_exh_cyl_out__0,pd_air_ic__0,pr_exh_turb_out__0,te_air_ic_out__0,te_seawater,te_air_comp_in_a__0,te_air_comp_in_b__0,fr_tc__0,...,pr_cyl_max__0,se_mip__0,te_exh_cyl_out__0_1,fr_eng_setpoint,te_air_scav_rec_iso,pr_cyl_max_mv_iso,pr_cyl_comp_mv_iso,fr_eng_ecs,pr_air_scav_iso,engine_type_G80ME-C9.5-GI-LPSCR
count,316581,316581.000000,316581.000000,316581.000000,0.0,316581.000000,3.165810e+05,0.0,0.0,0.0,...,3.161550e+05,3.161550e+05,316581.000000,316581.000000,190526.000000,1.880240e+05,1.880240e+05,316581.000000,188024.000000,316581.0
mean,2024-03-04 03:39:59.767831552,0.832841,529.816610,3110.918122,NaN,307.181012,2.731500e+02,NaN,NaN,NaN,...,1.451855e+07,9.473236e+05,529.816610,0.832522,304.499911,1.456114e+07,1.077625e+07,0.832841,99833.635388,1.0
min,2023-10-01 05:00:00,0.167429,319.150000,0.000000,NaN,293.350000,2.731500e+02,NaN,NaN,NaN,...,3.837725e+06,-9.958676e+04,319.150000,0.000000,298.952784,4.026332e+06,4.055897e+06,0.167429,597.633930,1.0
25%,2023-12-12 22:19:00,0.777719,513.150000,1770.000000,NaN,305.850000,2.731500e+02,NaN,NaN,NaN,...,1.282315e+07,7.901175e+05,513.150000,0.778309,302.687664,1.300431e+07,9.346316e+06,0.777719,60194.674426,1.0
50%,2024-03-05 10:31:00,0.897132,542.150000,3290.000000,NaN,307.150000,2.731500e+02,NaN,NaN,NaN,...,1.566857e+07,1.045047e+06,542.150000,0.899969,303.826959,1.551455e+07,1.157711e+07,0.897132,106705.193931,1.0
75%,2024-05-13 19:57:00,0.917471,553.150000,4370.000000,NaN,308.150000,2.731500e+02,NaN,NaN,NaN,...,1.637930e+07,1.121625e+06,553.150000,0.916650,307.118440,1.628848e+07,1.227609e+07,0.917471,131549.740285,1.0
max,2024-07-31 16:38:00,1.022311,597.150000,10000.000000,NaN,318.750000,2.731500e+02,NaN,NaN,NaN,...,1.891545e+07,1.524548e+06,597.150000,1.016633,310.687664,1.909924e+07,1.560651e+07,1.022311,229136.394211,1.0
std,NaN,0.142793,39.822783,1807.113916,NaN,1.789116,1.705305e-13,NaN,NaN,NaN,...,2.561383e+06,2.658676e+05,39.822783,0.143734,2.474008,2.466624e+06,2.250869e+06,0.142793,52355.439774,0.0


In [ ]:
df1.describe()

,time,fr_eng,te_exh_cyl_out__0,pd_air_ic__0,pr_exh_turb_out__0,te_air_ic_out__0,te_seawater,te_air_comp_in_a__0,te_air_comp_in_b__0,fr_tc__0,...,pr_cyl_max__0,se_mip__0,te_exh_cyl_out__0_1,fr_eng_setpoint,te_air_scav_rec_iso,pr_cyl_max_mv_iso,pr_cyl_comp_mv_iso,fr_eng_ecs,pr_air_scav_iso,engine_type_G80ME-C9.5-GI-LPSCR
count,105527,105527.000000,105527.000000,105527.000000,0.0,105527.000000,1.055270e+05,0.0,0.0,0.0,...,1.053750e+05,1.053750e+05,105527.000000,105527.000000,63500.000000,6.266600e+04,6.266600e+04,105527.000000,62666.000000,105527.0
mean,2024-03-04 03:39:57.460175872,0.832848,529.814569,3111.361926,NaN,307.180843,2.731500e+02,NaN,NaN,NaN,...,1.451877e+07,9.473490e+05,529.814569,0.832521,304.499043,1.456122e+07,1.077643e+07,0.832848,99836.699333,1.0
min,2023-10-01 05:01:00,0.167429,321.150000,0.000000,NaN,293.350000,2.731500e+02,NaN,NaN,NaN,...,3.837725e+06,-9.958676e+04,321.150000,0.000000,298.952784,4.027020e+06,4.056834e+06,0.167429,3098.270339,1.0
25%,2023-12-12 22:19:30,0.777576,513.150000,1770.000000,NaN,305.850000,2.731500e+02,NaN,NaN,NaN,...,1.282562e+07,7.904133e+05,513.150000,0.778309,302.687664,1.300006e+07,9.345829e+06,0.777576,60206.960626,1.0
50%,2024-03-05 10:31:00,0.897077,542.150000,3290.000000,NaN,307.150000,2.731500e+02,NaN,NaN,NaN,...,1.566906e+07,1.045234e+06,542.150000,0.899969,303.826959,1.551538e+07,1.157662e+07,0.897077,106687.706718,1.0
75%,2024-05-13 19:56:30,0.917473,553.150000,4370.000000,NaN,308.150000,2.731500e+02,NaN,NaN,NaN,...,1.637984e+07,1.121550e+06,553.150000,0.916647,307.118440,1.628844e+07,1.227628e+07,0.917473,131524.664414,1.0
max,2024-07-31 16:37:00,1.020961,597.150000,10000.000000,NaN,317.350000,2.731500e+02,NaN,NaN,NaN,...,1.891545e+07,1.524548e+06,597.150000,1.016633,310.687664,1.909924e+07,1.560651e+07,1.020961,227660.755509,1.0
std,NaN,0.142778,39.830309,1805.671624,NaN,1.788752,5.684369e-14,NaN,NaN,NaN,...,2.561205e+06,2.658468e+05,39.830309,0.143739,2.473322,2.465755e+06,2.250138e+06,0.142778,52343.895098,0.0
